In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import bs4 as bs
import requests
import re
import json

In [2]:
# Sportsbet
hdr = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
sb = requests.get("https://www.sportsbet.com.au/betting/ufc-mma", headers = hdr).text
soup = bs.BeautifulSoup(sb, 'html.parser')
sb_containers = soup.find_all("div", {"class": "marketRow_f4jyqbz"})
# Initialize Datasets
fight_time = []
fighter_A = []
fighter_B = []
bet_A = []
bet_B = []

for container in sb_containers:
    names = container.find_all('span', {'data-automation-id' : re.compile('.*two-outcome-outcome-name')})
    bets = container.find_all('span', {'data-automation-id' : 'price-text'})
    if names[0].string < names[1].string:
        fighter_A.append(names[0].string)
        bet_A.append(bets[0].string)
        fighter_B.append(names[1].string)
        bet_B.append(bets[1].string)
    else:
        fighter_B.append(names[0].string)
        bet_B.append(bets[0].string)
        fighter_A.append(names[1].string)
        bet_A.append(bets[1].string)

sports_bet = pd.DataFrame({'fighter_A' : fighter_A, 'fighter_B' : fighter_B, 'bet_A' : bet_A, 'bet_B' : bet_B})
sports_bet['Site'] = "Sportsbet"
sports_bet

,fighter_A,fighter_B,bet_A,bet_B,Site
0,Abdul Razak Alhassan,Joaquin Buckley,2.31,1.59,Sportsbet
1,Bill Algeo,Joanderson Brito,1.94,1.86,Sportsbet
2,Brandon Royval,Rogerio Bontorin,1.54,2.43,Sportsbet
3,Brian Kelleher,Saidyokub Kakhramonov,2.25,1.63,Sportsbet
4,Caio Borralho,Jamie Pickett,1.34,3.18,Sportsbet
5,Chase Sherman,Jake Collier,2.10,1.72,Sportsbet
6,Court McGee,Ramiz Brahimaj,1.90,1.90,Sportsbet
7,Dakota Bush,Viacheslav Borshchev,2.72,1.44,Sportsbet
8,Gabriel Benitez,T.J Brown,1.44,2.73,Sportsbet
9,Jennifer Maia,Katlyn Chookagian,2.31,1.59,Sportsbet


In [3]:
# Unibet
ub = requests.get("https://www.unibet.com.au/sportsbook-feeds/views/filter/ufc_mma/matches").json()
ub_parsed = ub['layout']['sections'][1]['widgets'][0]['matches']['groups'][0]['events']

# Initialize Datasets
fight_time = []
fighter_A = []
fighter_B = []
bet_A = []
bet_B = []

# Insert Datasets
for fight in ub_parsed:
    try:
        a = fight['mainBetOffer']
        b = fight['mainBetOffer']
    except:
        continue
    fight_time.append(fight['event']['start'])
    name_A_split = fight['event']['homeName'].split(',')
    if len(name_A_split) == 1:
        name_A = name_A_split[0].strip()
    else:
        name_A = name_A_split[1].strip() + " " + name_A_split[0].strip()
    dbet_A = fight['mainBetOffer']['outcomes'][0]['oddsDecimal']
    name_B_split = fight['event']['awayName'].split(',')
    if len(name_B_split) == 1:
        name_B = name_B_split[0].strip()
    else:
        name_B = name_B_split[1].strip() + " " + name_B_split[0].strip()
    dbet_B = fight['mainBetOffer']['outcomes'][1]['oddsDecimal']
    if name_A < name_B:
        fighter_A.append(name_A)
        bet_A.append(dbet_A)
        fighter_B.append(name_B)
        bet_B.append(dbet_B)
    else:
        fighter_B.append(name_A)
        bet_B.append(dbet_A)
        fighter_A.append(name_B)
        bet_A.append(dbet_B)

unibet = pd.DataFrame({'fight_time': fight_time, 'fighter_A' : fighter_A, 'fighter_B' : fighter_B, 'bet_A' : bet_A, 'bet_B' : bet_B})
unibet['Site'] = 'Unibet'
unibet

,fight_time,fighter_A,fighter_B,bet_A,bet_B,Site
0,2022-01-15T22:00:00Z,Abdul Alhassan Razak,Joaquin Buckley,2.15,1.70,Unibet
1,2022-01-15T22:00:00Z,Michel Pereira,Muslim Salikhov,1.80,2.00,Unibet
2,2022-01-15T22:00:00Z,Calvin Kattar,Giga Chikadze,2.70,1.47,Unibet
3,2022-01-15T22:00:00Z,Brandon Royval,Rogerio Bontorin,1.57,2.40,Unibet
4,2022-01-15T22:00:00Z,Kleydson Rodrigues,Zarrukh Adashev,1.38,3.05,Unibet
5,2022-01-15T22:00:00Z,Chase Sherman,Jake Collier,2.00,1.80,Unibet
6,2022-01-15T22:00:00Z,Dakota Bush,Viacheslav Borshchev,2.60,1.50,Unibet
7,2022-01-15T22:00:00Z,Gabriel Benitez,TJ Brown,1.45,2.75,Unibet
8,2022-01-15T22:00:00Z,Jennifer Maia,Katlyn Chookagian,2.30,1.62,Unibet
9,2022-01-15T22:00:00Z,Bill Algeo,Joanderson Brito,2.00,1.80,Unibet


In [4]:
# Neds
neds = requests.get("https://api.neds.com.au/v2/sport/event-request?category_ids=%5B%222768e4b7-effa-4bd1-929d-2e27f46af4f6%22%5D").json()

fighters_id = []
fighters_name = []
fighters_market_id = []

for key, fighters_info in neds['entrants'].items():
    fighters_id.append(key)
    fighters_name.append(fighters_info['name'])
    fighters_market_id.append(fighters_info['market_id'])

ned_fighters = pd.DataFrame({'fighters_id' : fighters_id, 'fighters_name' : fighters_name, 'fighters_market_id' : fighters_market_id})

fighters_id = []
fighters_odds = []

for key, odds in neds['prices'].items():
    fighters_id.append(key.split(':')[0])
    fighters_odds.append(1 + (odds['odds']['numerator'] / odds['odds']['denominator']))

ned_fighters_odds = pd.DataFrame({'fighters_id' : fighters_id, 'fighters_odds' : fighters_odds})

fighters_market_id = []
fight_time = []

for key, event_info in neds['events'].items():
    fighters_market_id.append(event_info['main_markets'][0])
    fight_time.append(event_info['actual_start'])

ned_fights = pd.DataFrame({'fighters_market_id' : fighters_market_id, 'fight_time' : fight_time})

neds = ned_fights.merge(ned_fighters, on = 'fighters_market_id').groupby(['fighters_market_id', 'fight_time']).agg({'fighters_name' : ['min', 'max']})
neds = neds.reset_index()
neds.columns = ['fighters_market_id', 'fight_time', 'fighter_A', 'fighter_B']
neds = neds.merge(ned_fighters, how = 'left', left_on = ['fighter_A', 'fighters_market_id'], right_on = ['fighters_name', 'fighters_market_id']).drop(columns = 'fighters_name')
neds = neds.merge(ned_fighters, how = 'left', left_on = ['fighter_B', 'fighters_market_id'], right_on = ['fighters_name', 'fighters_market_id']).drop(columns = 'fighters_name')
neds = neds.rename(columns = {'fighters_id_x': 'fighters_id_A', 'fighters_id_y': 'fighters_id_B'})
neds = neds.merge(ned_fighters_odds, how = 'left', left_on = 'fighters_id_A', right_on = 'fighters_id').drop(columns = 'fighters_id')
neds = neds.merge(ned_fighters_odds, how = 'left', left_on = 'fighters_id_B', right_on = 'fighters_id').drop(columns = 'fighters_id')
neds = neds.rename(columns = {'fighters_odds_x': 'bet_A', 'fighters_odds_y': 'bet_B'})
neds = neds.drop(columns = ['fighters_market_id', 'fighters_id_A', 'fighters_id_B'])
neds['Site'] = "Neds"
neds


,fight_time,fighter_A,fighter_B,bet_A,bet_B,Site
0,2022-01-23T03:00:00Z,Ciryl Gane,Francis Ngannou,1.90,1.90,Neds
1,2022-01-16T04:00:00Z,Abdul Razak Alhassan,Joaquin Buckley,2.25,1.67,Neds
2,2022-01-16T04:00:00Z,Chase Sherman,Jake Collier,2.02,1.80,Neds
3,2022-01-23T03:00:00Z,Saimon Oliveira,Tony Gravely,2.75,1.46,Neds
4,2022-01-23T03:00:00Z,Aleksei Oleinik,Greg Hardy,2.35,1.60,Neds
5,2022-01-23T03:00:00Z,Cody Stamann,Said Nurmagomedov,2.55,1.52,Neds
6,2022-01-23T03:00:00Z,Rodolfo Vieira,Wellington Turman,1.38,3.05,Neds
7,2022-01-16T04:00:00Z,Gabriel Benitez,TJ Brown,1.53,2.55,Neds
8,2022-01-23T03:00:00Z,Ji Yeon Kim,Poliana Botelho,1.73,2.10,Neds
9,2022-01-16T04:00:00Z,Dakota Bush,Viacheslav Borshchev,2.55,1.52,Neds


In [5]:
# LadBrokes
ladbrokes = requests.get("https://api.ladbrokes.com.au/v2/sport/event-request?category_ids=%5B%222768e4b7-effa-4bd1-929d-2e27f46af4f6%22%5D").json()

fighters_id = []
fighters_name = []
fighters_market_id = []

for key, fighters_info in ladbrokes['entrants'].items():
    fighters_id.append(key)
    fighters_name.append(fighters_info['name'])
    fighters_market_id.append(fighters_info['market_id'])

ned_fighters = pd.DataFrame({'fighters_id' : fighters_id, 'fighters_name' : fighters_name, 'fighters_market_id' : fighters_market_id})

fighters_id = []
fighters_odds = []

for key, odds in ladbrokes['prices'].items():
    fighters_id.append(key.split(':')[0])
    fighters_odds.append(1 + (odds['odds']['numerator'] / odds['odds']['denominator']))

ned_fighters_odds = pd.DataFrame({'fighters_id' : fighters_id, 'fighters_odds' : fighters_odds})

fighters_market_id = []
fight_time = []

for key, event_info in ladbrokes['events'].items():
    fighters_market_id.append(event_info['main_markets'][0])
    fight_time.append(event_info['actual_start'])

ned_fights = pd.DataFrame({'fighters_market_id' : fighters_market_id, 'fight_time' : fight_time})

ladbrokes = ned_fights.merge(ned_fighters, on = 'fighters_market_id').groupby(['fighters_market_id', 'fight_time']).agg({'fighters_name' : ['min', 'max']})
ladbrokes = ladbrokes.reset_index()
ladbrokes.columns = ['fighters_market_id', 'fight_time', 'fighter_A', 'fighter_B']
ladbrokes = ladbrokes.merge(ned_fighters, how = 'left', left_on = ['fighter_A', 'fighters_market_id'], right_on = ['fighters_name', 'fighters_market_id']).drop(columns = 'fighters_name')
ladbrokes = ladbrokes.merge(ned_fighters, how = 'left', left_on = ['fighter_B', 'fighters_market_id'], right_on = ['fighters_name', 'fighters_market_id']).drop(columns = 'fighters_name')
ladbrokes = ladbrokes.rename(columns = {'fighters_id_x': 'fighters_id_A', 'fighters_id_y': 'fighters_id_B'})
ladbrokes = ladbrokes.merge(ned_fighters_odds, how = 'left', left_on = 'fighters_id_A', right_on = 'fighters_id').drop(columns = 'fighters_id')
ladbrokes = ladbrokes.merge(ned_fighters_odds, how = 'left', left_on = 'fighters_id_B', right_on = 'fighters_id').drop(columns = 'fighters_id')
ladbrokes = ladbrokes.rename(columns = {'fighters_odds_x': 'bet_A', 'fighters_odds_y': 'bet_B'})
ladbrokes = ladbrokes.drop(columns = ['fighters_market_id', 'fighters_id_A', 'fighters_id_B'])
ladbrokes['Site'] = "Ladbrokes"
ladbrokes

,fight_time,fighter_A,fighter_B,bet_A,bet_B,Site
0,2022-01-23T03:00:00Z,Ciryl Gane,Francis Ngannou,1.90,1.90,Ladbrokes
1,2022-01-16T04:00:00Z,Abdul Razak Alhassan,Joaquin Buckley,2.25,1.67,Ladbrokes
2,2022-01-16T04:00:00Z,Chase Sherman,Jake Collier,2.02,1.80,Ladbrokes
3,2022-01-23T03:00:00Z,Saimon Oliveira,Tony Gravely,2.75,1.46,Ladbrokes
4,2022-01-23T03:00:00Z,Aleksei Oleinik,Greg Hardy,2.35,1.60,Ladbrokes
5,2022-01-23T03:00:00Z,Cody Stamann,Said Nurmagomedov,2.55,1.52,Ladbrokes
6,2022-01-23T03:00:00Z,Rodolfo Vieira,Wellington Turman,1.38,3.05,Ladbrokes
7,2022-01-16T04:00:00Z,Gabriel Benitez,TJ Brown,1.53,2.55,Ladbrokes
8,2022-01-23T03:00:00Z,Ji Yeon Kim,Poliana Botelho,1.73,2.10,Ladbrokes
9,2022-01-16T04:00:00Z,Dakota Bush,Viacheslav Borshchev,2.55,1.52,Ladbrokes


In [6]:
#Bluebet
bb = requests.get("https://web20-api.bluebet.com.au/MasterCategory?EventTypeId=128&WithLevelledMarkets=true&format=json").json()
leagues = bb['MasterCategories']

# Initialize Datasets
fight_time = []
fighter_A = []
fighter_B = []
bet_A = []
bet_B = []

for league in leagues:
    fights = league['Categories'][0]['MasterEvents']
    for fight in fights:
        time = fight['MaxAdvertisedStartTime'].replace("/Date(","").replace(")/","")
        fight_time.append(time)
        f_A = fight['Markets'][0]['OutcomeName']
        f_B = fight['Markets'][1]['OutcomeName']
        b_A = fight['Markets'][0]['Price']
        b_B = fight['Markets'][1]['Price']

        if f_A < f_B:
            fighter_A.append(f_A)
            bet_A.append(b_A)
            fighter_B.append(f_B)
            bet_B.append(b_B)
        else:
            fighter_B.append(f_A)
            bet_B.append(b_A)
            fighter_A.append(f_B)
            bet_A.append(b_B)

fight_time = pd.to_datetime(fight_time)
bluebet = pd.DataFrame({'fight_time': fight_time, 'fighter_A' : fighter_A, 'fighter_B' : fighter_B, 'bet_A' : bet_A, 'bet_B' : bet_B})
bluebet['Site'] = 'Bluebet'
bluebet

,fight_time,fighter_A,fighter_B,bet_A,bet_B,Site
0,2022-01-15 20:00:00+00:00,Brian Kelleher,Saidyokub Kakhramonov,2.25,1.62,Bluebet
1,2022-01-15 20:20:00+00:00,Dakota Bush,Viacheslav Borshchev,2.65,1.46,Bluebet
2,2022-01-15 22:00:00+00:00,Michel Pereira,Muslim Salikhov,1.78,2.00,Bluebet
3,2022-01-15 22:20:00+00:00,Calvin Kattar,Giga Chikadze,2.75,1.43,Bluebet
4,2022-01-23 03:00:00+00:00,Brandon Moreno,Deiveson Figueiredo,1.55,2.40,Bluebet
5,2022-01-23 03:00:00+00:00,Ciryl Gane,Francis Ngannou,1.88,1.88,Bluebet
6,2022-02-12 23:00:00+00:00,Israel Adesanya,Robert Whittaker,1.43,2.75,Bluebet


In [7]:
# TAB
tab = requests.get("https://api.beta.tab.com.au/v1/recommendation-service/UFC/featured?jurisdiction=NSW").json()
competition = tab['competitions']

# Initialize Datasets
fight_time = []
fighter_A = []
fighter_B = []
bet_A = []
bet_B = []

competition_type = []

for comp in competition:
    if len(comp['tournaments']) >= 1:
        competition_type.append('tournament')
    else:
        competition_type.append('matches')

for i, c in enumerate(competition_type):
    if c == 'matches':
        comp = competition[i]['matches']
        for match in comp:
            time = match['startTime']
            fight_time.append(time)
            name_A_split = re.search('([A-Z].*) ([A-Z].*)*', match['markets'][0]['propositions'][0]['name'])
            name_B_split = re.search('([A-Z].*) ([A-Z].*)*', match['markets'][0]['propositions'][1]['name'])
            if name_A_split != None:
                f_A = name_A_split.group(2) + " " + name_A_split.group(1).title()
            else:
                f_A = ""
            if name_B_split != None:
                f_B = name_B_split.group(2) + " " + name_B_split.group(1).title()
            else:
                f_B = ""
            b_A = match['markets'][0]['propositions'][0]['returnWin']
            b_B = match['markets'][0]['propositions'][1]['returnWin']
            
            if f_A < f_B:
                fighter_A.append(f_A)
                bet_A.append(b_A)
                fighter_B.append(f_B)
                bet_B.append(b_B)
            else:
                fighter_B.append(f_A)
                bet_B.append(b_A)
                fighter_A.append(f_B)
                bet_A.append(b_B)
    else:
        comp = competition[i]['tournaments'][0]['matches']
        for match in comp:
            time = match['startTime']
            fight_time.append(time)
            name_A_split = re.search('([A-Z].*) ([A-Z].*)*', match['markets'][0]['propositions'][0]['name'])
            name_B_split = re.search('([A-Z].*) ([A-Z].*)*', match['markets'][0]['propositions'][1]['name'])
            f_A = name_A_split.group(2) + " " + name_A_split.group(1).title()
            f_B = name_B_split.group(2) + " " + name_B_split.group(1).title()
            b_A = match['markets'][0]['propositions'][0]['returnWin']
            b_B = match['markets'][0]['propositions'][1]['returnWin']
            
            if f_A < f_B:
                fighter_A.append(f_A)
                bet_A.append(b_A)
                fighter_B.append(f_B)
                bet_B.append(b_B)
            else:
                fighter_B.append(f_A)
                bet_B.append(b_A)
                fighter_A.append(f_B)
                bet_A.append(b_B)

tab = pd.DataFrame({'fight_time': fight_time, 'fighter_A' : fighter_A, 'fighter_B' : fighter_B, 'bet_A' : bet_A, 'bet_B' : bet_B})
tab['Site'] = 'TAB'
tab

,fight_time,fighter_A,fighter_B,bet_A,bet_B,Site
0,2022-01-23T01:00:00.000Z,Cyril Gane,Francis Ngannou,2.00,1.80,TAB
1,2022-01-15T20:00:00.000Z,Borshchev Slava,Bush Dakota,1.48,2.60,TAB
2,2022-01-15T20:00:00.000Z,Kakhramaonov Saidyokub,Kelleher Brian,1.60,2.30,TAB
3,2022-01-15T20:00:00.000Z,Adashev Zarrukh,Rodrigues Kleydson,3.00,1.38,TAB
4,2022-01-15T22:00:00.000Z,Chikadze Giga,Kattar Calvin,1.45,2.70,TAB
5,2022-01-15T22:00:00.000Z,Pereira Michael,Salikhov Muslim,1.75,2.05,TAB


In [8]:
# Points Bet
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
pb = requests.get("https://api.pointsbet.com/api/v2/competitions/7874/events/featured?includeLive=false&page=1",headers=headers).json()
fights = pb['events']

# Initialize Datasets
fight_time = []
fighter_A = []
fighter_B = []
bet_A = []
bet_B = []

for fight in fights:
    time = fight['startsAt']
    fight_time.append(time)
    f_A = fight['fixedOddsMarkets'][0]['outcomes'][0]['name']
    f_B = fight['fixedOddsMarkets'][0]['outcomes'][1]['name']
    b_A = fight['fixedOddsMarkets'][0]['outcomes'][0]['price']
    b_B = fight['fixedOddsMarkets'][0]['outcomes'][1]['price']

    if f_A < f_B:
        fighter_A.append(f_A)
        bet_A.append(b_A)
        fighter_B.append(f_B)
        bet_B.append(b_B)
    else:
        fighter_B.append(f_A)
        bet_B.append(b_A)
        fighter_A.append(f_B)
        bet_A.append(b_B)

pointsbet = pd.DataFrame({'fight_time': fight_time, 'fighter_A' : fighter_A, 'fighter_B' : fighter_B, 'bet_A' : bet_A, 'bet_B' : bet_B})
pointsbet['Site'] = 'Pointsbet'
pointsbet


,fight_time,fighter_A,fighter_B,bet_A,bet_B,Site
0,2022-01-15T20:00:00Z,Brian Kelleher,Saidyokub Kakhramonov,2.30,1.62,Pointsbet
1,2022-01-15T20:20:00Z,Dakota Bush,Viacheslav Borshchev,2.67,1.47,Pointsbet
2,2022-01-15T22:00:00Z,Michel Pereira,Muslim Salikhov,1.80,2.00,Pointsbet
3,2022-01-15T22:20:00Z,Calvin Kattar,Giga Chikadze,2.78,1.44,Pointsbet
4,2022-01-23T03:00:00Z,Brandon Moreno,Deiveson Figueiredo,1.57,2.40,Pointsbet
5,2022-01-23T03:00:00Z,Ciryl Gane,Francis Ngannou,1.90,1.90,Pointsbet
6,2022-02-12T23:00:00Z,Israel Adesanya,Robert Whittaker,1.44,2.78,Pointsbet


In [9]:
# For Group By Each Name
# TODO SITES
bet_union = pd.concat([sports_bet, unibet, neds, ladbrokes, tab, bluebet, pointsbet])
bet_union = bet_union[pd.to_numeric(bet_union['bet_A'], errors='coerce').notnull()]
bet_union = bet_union[pd.to_numeric(bet_union['bet_B'], errors='coerce').notnull()]
bet_union = bet_union[~bet_union['fighter_A'].str.contains("KO/DQ")]
bet_union = bet_union[~bet_union['fighter_B'].str.contains("KO/DQ")]
bet_union = bet_union[~bet_union['fighter_A'].str.contains("Any 2")]
bet_union = bet_union[~bet_union['fighter_B'].str.contains("Any 2")]
bet_union['bet_A'] = bet_union['bet_A'].apply(lambda x: float(x))
bet_union['bet_B'] = bet_union['bet_B'].apply(lambda x: float(x))
bet_union_temp_A = bet_union[['fighter_A', 'fighter_B', 'bet_A', 'Site']]
bet_union_temp_B = bet_union[['fighter_A', 'fighter_B', 'bet_B', 'Site']]
bet_union = bet_union.groupby(['fighter_A', 'fighter_B']).agg({'bet_A' : 'max', 'bet_B': 'max'})
bet_union = bet_union.merge(bet_union_temp_A, how = 'left', left_on = ['fighter_A', 'fighter_B', 'bet_A'], right_on = ['fighter_A', 'fighter_B', 'bet_A'])
bet_union = bet_union.merge(bet_union_temp_B, how = 'left', left_on = ['fighter_A', 'fighter_B', 'bet_B'], right_on = ['fighter_A', 'fighter_B', 'bet_B'])
bet_union['bet_A_implied'] = 1 / bet_union['bet_A']
bet_union['bet_B_implied'] = 1 / bet_union['bet_B']
bet_union['total_implied'] = bet_union['bet_A_implied'] + bet_union['bet_B_implied']
bet_union['profit'] = (1 - bet_union['total_implied']) * 100
bet_union['profit'] = bet_union['profit'].round(2)
bet_union['profitable'] = (bet_union['total_implied'] < 1)
pd.set_option('display.max_rows', 100)
bet_union[(bet_union['profitable'] == True) | (bet_union['total_implied'] < 1.02)]

,fighter_A,fighter_B,bet_A,bet_B,Site_x,Site_y,bet_A_implied,bet_B_implied,total_implied,profit,profitable
58,Gabriel Benitez,TJ Brown,1.53,2.75,Neds,Unibet,0.653595,0.363636,1.017231,-1.72,False
59,Gabriel Benitez,TJ Brown,1.53,2.75,Ladbrokes,Unibet,0.653595,0.363636,1.017231,-1.72,False
86,Kleydson Rodrigues,Zarrukh Adashev,1.42,3.25,Neds,Sportsbet,0.704225,0.307692,1.011918,-1.19,False
87,Kleydson Rodrigues,Zarrukh Adashev,1.42,3.25,Ladbrokes,Sportsbet,0.704225,0.307692,1.011918,-1.19,False


In [10]:
bet_union[(bet_union['Site_x'] == 'Neds')].sort_values('bet_B', ascending = 0)
bet_union[(bet_union['fighter_A'].str.contains('Paiva'))].sort_values('bet_B', ascending = 0)

,fighter_A,fighter_B,bet_A,bet_B,Site_x,Site_y,bet_A_implied,bet_B_implied,total_implied,profit,profitable


In [11]:
bet_union['diff'] = abs(bet_union['bet_A_implied'] - bet_union['bet_B_implied'])
# bet_union[bet_union['diff'] < 0.2].sort_values('diff', ascending = 1)

In [12]:
# bet_union[bet_union['diff'] > 0.5].sort_values('diff', ascending = 0)